In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_trans_sell = pd.read_csv("trans_sell.csv", encoding = "Big5")
df_trans_sell.columns = ["yyyymm", "id_number", "certificate","fund_id", "invest_configure", "surplus", "principal", "present_value"]

In [3]:
df_trans_sell['variation'] = df_trans_sell["present_value"] - df_trans_sell["principal"]
df_trans_sell['merge_key'] = df_trans_sell['yyyymm'].astype(str) + df_trans_sell['id_number'] + df_trans_sell['fund_id']

In [4]:
print(len(df_trans_sell[df_trans_sell['surplus'] > 0].index))
print(len(df_trans_sell))

434701
602973


In [5]:
# df_trans_sell['merge_key'].value_counts().head(100)

In [6]:
df_trans_sell[df_trans_sell['certificate'] == '10778G51500060']

,yyyymm,id_number,certificate,fund_id,invest_configure,surplus,principal,present_value,variation,merge_key
25160,201607,A1727691450,10778G51500060,78G,a.定時定額,56614.06818,11.832407,9.900586,-1.931822,201607A172769145078G
165869,201605,A1727691450,10778G51500060,78G,a.定時定額,-104033.81236,613549.724190,509515.911833,-104033.812357,201605A172769145078G


In [7]:
df_groupby = df_trans_sell.groupby(['merge_key']).agg({'variation':'sum', 'principal':'sum', 'present_value':'sum'})

In [8]:
df_groupby.loc['201701A1925995480224']

variation        7.093340e+06
principal        3.590971e+07
present_value    4.300305e+07
Name: 201701A1925995480224, dtype: float64

In [9]:
df_groupby.drop(df_groupby[df_groupby['principal'] == 0].index, inplace = True)
df_groupby[df_groupby['principal'] == 0 ]

,variation,principal,present_value
merge_key,,,


In [10]:
df_groupby['realized_gain/loss'] = df_groupby['variation']/df_groupby['principal']

In [11]:
df_groupby['realized_gain/loss'].value_counts().sort_index()

-1.000000     1
-0.959488     1
-0.890653     1
-0.872828     1
-0.868810     1
             ..
 6.392667     1
 6.682573     1
 6.723000     1
 13.648275    1
 19.473534    1
Name: realized_gain/loss, Length: 442248, dtype: int64

In [12]:
df_groupby = pd.DataFrame({'merge_key': df_groupby.index, 'realized_gain/loss': df_groupby['realized_gain/loss']})

In [13]:
df_groupby.reset_index(drop=True, inplace=True)

In [14]:
df_groupby

,merge_key,realized_gain/loss
0,20160189054091EH1K01,0.073765
1,201601A1220303130505,-0.215033
2,201601A1220335170396,-0.121000
3,201601A1220335170MB4,-0.044319
4,201601A1221932940596,-0.107574
...,...,...
527964,202012Z2222846790J84,0.045550
527965,202012Z2222846790MB0,0.065917
527966,202012Z2222853420FH5,0.123000
527967,202012Z2222853420FW1,0.120000


In [15]:
df_trans_buy = pd.read_csv("trans_buy.csv")
df_trans_buy.columns = ['id_number', 'certificate', 'fund_id', 'buy_date','deduction_num', 'deduction_local_amount']
top_client = df_trans_buy['id_number'].value_counts().head(100).index
df_trans_sell[df_trans_sell['id_number'].isin(top_client)]

,yyyymm,id_number,certificate,fund_id,invest_configure,surplus,principal,present_value,variation,merge_key
11,201601,A1757969050,138F8741500438,F87,b.單筆申購,-10431.97276,167997.500000,157565.527240,-10431.972760,201601A1757969050F87
111,201601,A2791533910,126AE751500084,AE7,a.定時定額,-5233.70550,119700.937298,114467.231794,-5233.705505,201601A2791533910AE7
157,201601,A1924415990,105CK331300026,CK3,a.定時定額,23.00000,12000.000000,12023.000000,23.000000,201601A1924415990CK3
230,201601,A1924415990,105J0111500151,J01,a.定時定額,486.00000,12000.000000,12486.000000,486.000000,201601A1924415990J01
231,201601,A1924415990,105Y3251300200,Y32,a.定時定額,-2585.92154,36129.304800,33543.383262,-2585.921538,201601A1924415990Y32
...,...,...,...,...,...,...,...,...,...,...
602927,202012,A2732223460,190J8002000236,J80,b.單筆申購,1805.00000,100000.000000,101805.000000,1805.000000,202012A2732223460J80
602928,202012,A2732223460,190MU702000156,MU7,b.單筆申購,2913.00000,100000.000000,102913.000000,2913.000000,202012A2732223460MU7
602955,202012,A2795667510,039MX452000033,MX4,a.定時定額,2278.62111,45758.208000,48036.829104,2278.621104,202012A2795667510MX4
602969,202012,C1726080360,190J8412000150,J84,a.定時定額,3.00000,100.000000,103.000000,3.000000,202012C1726080360J84
